In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Mount your Google Drive in Google Colab.
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# csv_path = '/content/drive/My Drive/Open IIT Data Team : Not ARIMA/Shimla.csv'
# df = pd.read_csv(csv_path)

In [ ]:
df = pd.read_csv('Shimla.csv')

In [ ]:
df.head(50)

In [ ]:
X=df.iloc[:,1:]
X.columns=df.columns[1:]
sorted_columns = sorted(X.columns)
X = X.reindex(columns=sorted_columns)
X=X.iloc[:,:-1]

y=df.iloc[:,1]

In [ ]:
X.head()

In [ ]:
X.dtypes

In [ ]:
XTrain=X.iloc[:12,:]
XTrain.columns=X.columns

YTrain=y.iloc[:12]

In [ ]:
XTrain.head()

In [ ]:
print(XTrain.shape)
print(YTrain.shape)

### Plotting features

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(12, 8))
axes = axes.ravel()
for i, col in enumerate(XTrain.columns):
    ax = axes[i]
    ax.plot(XTrain.index, XTrain[col])
    ax.set_title(f'{XTrain.columns[i]}')
    ax.set_xlabel('Index')
    ax.set_ylabel(col)
    
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have XTrain and y_train as pandas DataFrames or Series

# Create subplots for XTrain columns
fig, axes = plt.subplots(4, 3, figsize=(12, 8))
axes = axes.ravel()
for i, col in enumerate(XTrain.columns):
    ax = axes[i]
    ax.plot(XTrain.index, XTrain[col])
    ax.set_title(f'{XTrain.columns[i]}')
    ax.set_xlabel('Index')
    ax.set_ylabel(col)

# Add a subplot for y_train
ax = axes[-1]
ax.plot(XTrain.index, YTrain, color='red')  # Adjust the color as needed
ax.set_title('y_train')
ax.set_xlabel('Index')
ax.set_ylabel('y_train')

plt.tight_layout()
plt.show()


In [ ]:
column_to_drop = "Monthly Data"  
XTrain = XTrain.drop(column_to_drop, axis=1)

In [ ]:
corrmat = XTrain.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corrmat, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
XTrain_normalized = scaler.fit_transform(XTrain)

In [ ]:
y.shape

In [ ]:
# data_reshaped = y.to_numpy().reshape(13, 12)

# Calculate the average for each group of 12 indexes (across each row)
# averages = data_reshaped.mean(axis=1)

# Create an array for the x-axis (assuming each group represents a time period)
x = np.arange(1, 157)  # Assuming 13 groups

# Plot the averages
plt.plot(x, y, marker='o')
plt.xlabel('Time Period')
plt.ylabel('Average Value')
plt.title('Average of Every 12 Indexes')
plt.grid(True)
plt.show()


In [ ]:
data_reshaped = y.to_numpy().reshape(13, 12)

# Calculate the average for each group of 12 indexes (across each row)
averages = data_reshaped.mean(axis=1)

# Create an array for the x-axis (assuming each group represents a time period)
x = np.arange(1, 14)  # Assuming 13 groups

# Plot the averages
plt.plot(x, averages, marker='o')
plt.xlabel('Time Period')
plt.ylabel('Average Value')
plt.title('Average of Every 12 Indexes')
plt.grid(True)
plt.show()


### Model Selection

In [ ]:
X=df.iloc[:,1:]
X.columns=df.columns[1:]
sorted_columns = sorted(X.columns)
X = X.reindex(columns=sorted_columns)
column_to_drop = "Monthly Data"  
X = X.drop(column_to_drop, axis=1)
X=X.iloc[:,:-1]

y=df.iloc[:,0]

In [ ]:
X_reshaped=X.to_numpy().reshape(13,12,11)
y_reshaped=y.to_numpy().reshape(13,12)

In [ ]:
X.head()

In [ ]:
y

In [ ]:
print(X_reshaped.shape)
print(y_reshaped.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.LSTM(50, input_shape=(12, 11), return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(12)))  
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_reshaped, y_reshaped, epochs=100, batch_size=32, validation_data=(X_reshaped[0], y_reshaped[0]))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.LSTM(100, input_shape=(12, 11), return_sequences=True),
    layers.LSTM(100, return_sequences=True),
    layers.Dense(1)  # Adjust the output layer based on your problem
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_reshaped, y_reshaped, epochs=100, batch_size=32, validation_data=(X_reshaped, y_reshaped))

In [ ]:
y_pred = model.predict(X_reshaped[0].reshape(1,12,11))

In [ ]:
y_pred

In [ ]:
y_pred.shape

In [ ]:
y_reshaped[0].shape

In [ ]:
y_reshaped[0]

In [ ]:
import matplotlib.pyplot as plt
x = range(12)
plt.plot(x, y_reshaped[0], label='Actual', color='blue')
plt.plot(x, y_pred.reshape(12), label='Predicted', color='red')

# Add labels and a legend
plt.xlabel('Time or Data Points')
plt.ylabel('Values')
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Evaluate the model
test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)

In [ ]:
# Evaluate the model on the testing data
loss = model.evaluate(XTrain, YTrain)
print("Test Loss:", loss)